In [22]:
import pandas as pd
import numpy as np
import psycopg2
import boto3
import configparser
import seaborn as sb

## Configuración de la base de datos

In [15]:
config = configparser.ConfigParser()
config.read('proyecto.cfg')

['proyecto.cfg']

## Conexión a base de datos

In [16]:
import sql_queries

try:
    db_conn = psycopg2.connect(
        database = config.get('RDS', 'DB_NAME'),
        user = config.get('RDS', 'DB_USER'),
        password = config.get('RDS', 'DB_PASSWORD'),
        host = config.get('RDS', 'DB_HOST'),
        port = config.get('RDS', 'DB_PORT')
    )
    cursor = db_conn.cursor()
    cursor.execute(sql_queries.DDL_QUERY)
    db_conn.commit()
    print("Base de datos creada exitosamente.")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Base de datos creada exitosamente.


## Carga de datos

In [17]:
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{config.get('RDS','DB_HOST')}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

In [18]:
import csv

sql_data = [*csv.DictReader(open('Video_Games_Sales_as_at_22_Dec_2016.csv'))]

insertDataToSQL(
    sql_data,
    'gamesales'
)

Se han insertado 719 nuevos registros.


## Análisis de los datos

In [38]:
sql_query = 'SELECT * FROM gamesales'
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{config.get('RDS','DB_HOST')}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""

#Carga de los datos a un dataframe
gamesales = pd.read_sql(sql_query, postgres_driver)
gamesales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Game_id          16719 non-null  object
 1   Name             16719 non-null  object
 2   Platform         16719 non-null  object
 3   Year_of_Release  16719 non-null  object
 4   Genre            16719 non-null  object
 5   Publisher        16719 non-null  object
 6   NA_Sales         16719 non-null  object
 7   EU_Sales         16719 non-null  object
 8   JP_Sales         16719 non-null  object
 9   Other_Sales      16719 non-null  object
 10  Global_Sales     16719 non-null  object
 11  Critic_Score     16719 non-null  object
 12  Critic_Count     16719 non-null  object
 13  User_Score       16719 non-null  object
 14  User_Count       16719 non-null  object
 15  Developer        16719 non-null  object
 16  Rating           16719 non-null  object
dtypes: object(17)
memory usage: 2.2

### Medias de ventas por región
* Plataforma

In [57]:
#NA_Sales
plataforma = gamesales[['Platform','NA_Sales']]
plataforma['NA_Sales'] = plataforma['NA_Sales'].astype('float')
plataforma.groupby('Platform').sum().sort_values('NA_Sales',ascending=False)

C:\Users\ccast\AppData\Local\Temp\ipykernel_11408\2466501684.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plataforma['NA_Sales'] = plataforma['NA_Sales'].astype('float')


,NA_Sales
Platform,
X360,602.47
PS2,583.84
Wii,496.90
PS3,393.49
DS,382.67
PS,336.52
GBA,187.54
XB,186.69
N64,139.02


In [58]:
#EU_Sales
plataforma = gamesales[['Platform','EU_Sales']]
plataforma['EU_Sales'] = plataforma['EU_Sales'].astype('float')
plataforma.groupby('Platform').sum().sort_values('EU_Sales',ascending=False)

C:\Users\ccast\AppData\Local\Temp\ipykernel_11408\4190940016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plataforma['EU_Sales'] = plataforma['EU_Sales'].astype('float')


,EU_Sales
Platform,
PS2,339.29
PS3,330.29
X360,270.76
Wii,262.21
PS,213.61
DS,188.89
PC,142.44
PS4,141.09
GBA,75.25


In [59]:
#JP_Sales
plataforma = gamesales[['Platform','JP_Sales']]
plataforma['JP_Sales'] = plataforma['JP_Sales'].astype('float')
plataforma.groupby('Platform').sum().sort_values('JP_Sales',ascending=False)

C:\Users\ccast\AppData\Local\Temp\ipykernel_11408\2966933929.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plataforma['JP_Sales'] = plataforma['JP_Sales'].astype('float')


,JP_Sales
Platform,
DS,175.57
PS,139.82
PS2,139.20
SNES,116.55
3DS,100.67
NES,98.65
GB,85.12
PS3,80.19
PSP,76.78


In [60]:
#Other_Sales
plataforma = gamesales[['Platform','Other_Sales']]
plataforma['Other_Sales'] = plataforma['Other_Sales'].astype('float')
plataforma.groupby('Platform').sum().sort_values('Other_Sales',ascending=False)

C:\Users\ccast\AppData\Local\Temp\ipykernel_11408\1416699614.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plataforma['Other_Sales'] = plataforma['Other_Sales'].astype('float')


,Other_Sales
Platform,
PS2,193.44
PS3,135.68
X360,85.76
Wii,79.07
DS,59.28
PS4,48.35
PSP,41.42
PS,40.91
PC,22.38


In [61]:
#Global_Sales
plataforma = gamesales[['Platform','Global_Sales']]
plataforma['Global_Sales'] = plataforma['Global_Sales'].astype('float')
plataforma.groupby('Platform').sum().sort_values('Global_Sales',ascending=False)

C:\Users\ccast\AppData\Local\Temp\ipykernel_11408\594456332.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plataforma['Global_Sales'] = plataforma['Global_Sales'].astype('float')


,Global_Sales
Platform,
PS2,1255.64
X360,971.63
PS3,939.43
Wii,908.13
DS,807.10
PS,730.68
GBA,318.50
PS4,314.23
PSP,294.30
